In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from greenseer.configuration import do_global_configuration,get_global_configuration
do_global_configuration("config.properties")

In [3]:
from greenseer.repository.china_stock import  create_china_stock_assert_repository
from logging.config import fileConfig
fileConfig('logging_config.ini')
china_daily_repository =  create_china_stock_assert_repository()
data = china_daily_repository.load_data("600096").T

for column in  data.columns:
    print(column+"\n")

一年内到期的非流动负债(万元)

一年内到期的非流动资产(万元)

一般风险准备(万元)

专项储备(万元)

专项应付款(万元)

买入返售金融资产(万元)

交易性金融负债(万元)

交易性金融资产(万元)

代理买卖证券款(万元)

代理承销证券款(万元)

保险合同准备金(万元)

公益性生物资产(万元)

其他应交款(万元)

其他应付款(万元)

其他应收款(万元)

其他流动负债(万元)

其他流动资产(万元)

其他长期投资(万元)

其他非流动负债(万元)

其他非流动资产(万元)

内部应付款(万元)

内部应收款(万元)

减:库存股(万元)

卖出回购金融资产款(万元)

发放贷款及垫款(万元)

可供出售金融资产(万元)

向中央银行借款(万元)

吸收存款及同业存放(万元)

商誉(万元)

固定资产(万元)

固定资产净值(万元)

固定资产减值准备(万元)

固定资产原值(万元)

固定资产清理(万元)

国内票证结算(万元)

国际票证结算(万元)

在建工程(万元)

外币报表折算差额(万元)

存货(万元)

实收资本(或股本)(万元)

少数股东权益(万元)

工程物资(万元)

应交税费(万元)

应付保证金(万元)

应付债券(万元)

应付分保账款(万元)

应付利息(万元)

应付手续费及佣金(万元)

应付短期债券(万元)

应付票据(万元)

应付职工薪酬(万元)

应付股利(万元)

应付账款(万元)

应收保证金(万元)

应收保费(万元)

应收出口退税(万元)

应收分保合同准备金(万元)

应收分保账款(万元)

应收利息(万元)

应收票据(万元)

应收股利(万元)

应收补贴款(万元)

应收账款(万元)

开发支出(万元)

归属于母公司股东权益合计(万元)

待处理流动资产损益(万元)

待摊费用(万元)

所有者权益(或股东权益)合计(万元)

投资性房地产(万元)

拆入资金(万元)

拆出资金(万元)

拟分配现金股利(万元)

持有至到期投资(万元)

无形资产(万元)

未分配利润(万元)

未确定的投资损失(万元)

油气资产(万元)

流动负债合计(万元)

流动资产合计(万元)

生产性生物资产(万元)

盈余公积(万元)

短期借款(万元)

累计折旧(万元)

结算备付金(万